In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import csv 
from bs4 import BeautifulSoup
from multiprocessing import Pool
import os

url_list = "https://www.asus.com/us/Motherboards/AllProducts/"
driver = webdriver.PhantomJS('D:\\Github\\MovieCrawler\\phantomjs.exe')
# get()方法會一直等到頁面被完全加載，然後才會繼續程序
driver.get(url_list)

# 把selenium的webdriver调用page_source函数在传入BeautifulSoup中，就可以用BeautifulSoup解析网页了
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")


# 打印頁面標題：XXXX年最新电影_第1页-BT电影天堂
print(driver.title)

C:\Users\Kevin_Chiou\AppData\Local\Programs\Python\Python36-32\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


All Products | | ASUS USA


### 抓所有主板規格的URL

In [2]:
f = open('MB_SPEC_LINK.txt', 'w', encoding = 'UTF-8')    # 也可使用指定路徑等方式，如： C:\A.txt
div_list = soup.find_all('div', class_='product_level_1')
for div in div_list:
    li_list = div.find('ul', class_='unstyled all-model-list').contents
    for li in li_list:
        a = li.find('a')
        ##只要herf用中括號就可, 用herf = a.get_attribute('href')會跳error
        print(a['href'])
        f.write('https://www.asus.com' + a['href'] + 'specifications/' + '\n')
f.close() 


/us/Motherboards/H110S1-CSM/
/us/Motherboards/Q170M-CM-B-CSM-Pro/
/us/Motherboards/Q170T-CSM/
/us/Motherboards/H110I-PLUS-CSM/
/us/Motherboards/H110T-CSM/
/us/Motherboards/B150M-PLUS-D3/
/us/Motherboards/H110M-C-CSM/
/us/Motherboards/Q170M-C-CSM/
/us/Motherboards/PRIME-Q270M-C-CSM/
/us/Motherboards/PRIME-J3355I-C/
/us/Motherboards/PRIME-Z370-A/
/us/Motherboards/PRIME-Z370-P/
/us/Motherboards/EX-B250-V7/
/us/Motherboards/PRIME-B350M-E/
/us/Motherboards/PRIME-X399-A/
/us/Motherboards/PRIME-A320M-A/
/us/Motherboards/PRIME-A320M-C-R2-0/
/us/Motherboards/PRIME-X299-A/
/us/Motherboards/PRIME-X299-DELUXE/
/us/Motherboards/PRIME-H110M-P/
/us/Motherboards/PRIME-X370-A/
/us/Motherboards/PRIME-A320M-K/
/us/Motherboards/PRIME-B350-PLUS/
/us/Motherboards/PRIME-X370-PRO/
/us/Motherboards/PRIME-H270-PLUS-CSM/
/us/Motherboards/PRIME-B250M-C-CSM/
/us/Motherboards/PRIME-H270M-PLUS-CSM/
/us/Motherboards/PRIME-B250M-K/
/us/Motherboards/PRIME-B250M-A/
/us/Motherboards/PRIME-B250M-PLUS/
/us/Motherboards/PRI

### 改用Dictionary來存網址, 避免重複

In [25]:
Mb_URL_dic = {}
for div in div_list:
    li_list = div.find('ul', class_='unstyled all-model-list').contents
    for li in li_list:
        a = li.find('a')
        ##只要herf用中括號就可, 用herf = a.get_attribute('href')會跳error
        Mb_URL_dic[a.text] = ('https://www.asus.com' + a['href'] + 'specifications/')
Mb_URL_dic


{'B150M-A/M.2': 'https://www.asus.com/us/Motherboards/B150M-A-M-2/specifications/',
 'B150M-PLUS D3': 'https://www.asus.com/us/Motherboards/B150M-PLUS-D3/specifications/',
 'B250 MINING EXPERT': 'https://www.asus.com/us/Motherboards/B250-MINING-EXPERT/specifications/',
 'EX-B250-V7': 'https://www.asus.com/us/Motherboards/EX-B250-V7/specifications/',
 'H110I-PLUS/CSM': 'https://www.asus.com/us/Motherboards/H110I-PLUS-CSM/specifications/',
 'H110M-A/DP': 'https://www.asus.com/us/Motherboards/H110M-A-DP/specifications/',
 'H110M-A/M.2': 'https://www.asus.com/us/Motherboards/H110M-A-M-2/specifications/',
 'H110M-C/CSM': 'https://www.asus.com/us/Motherboards/H110M-C-CSM/specifications/',
 'H110M-E/M.2': 'https://www.asus.com/us/Motherboards/H110M-E-M-2/specifications/',
 'H110M-K': 'https://www.asus.com/us/Motherboards/H110M-K/specifications/',
 'H110S1/CSM': 'https://www.asus.com/us/Motherboards/H110S1-CSM/specifications/',
 'H110T/CSM': 'https://www.asus.com/us/Motherboards/H110T-CSM/spec

### 抓SPEC

In [26]:
SPEC_URL_list=[]
f = open("D:\\Github\\MovieCrawler\\MB_SPEC_LINK.txt","r", encoding = 'UTF-8')
for line in f:
    SPEC_URL_list.append(line[:-1])
f.close()

   

# url_list = "https://www.asus.com/us/Motherboards/H110S1-CSM/specifications/"
# driver.get(url_list)
# html = driver.page_source
# soup = BeautifulSoup(html, "html.parser")
# soup_title_list = soup.find_all('span', class_='spec-item')
# soup_data_list = soup.find_all('div', class_='spec-data')
# MB_name = soup.find('h1', class_='page-title').text

# title_list =['Board name']
# data_list = [MB_name]

# for title in soup_title_list:
#     title_list.append(title.text)
# for data in soup_data_list:
#     data_list.append(data.text)

## create DataFrame with title list
df = pd.DataFrame()
# df = df.append(pd.Series(data_list,index=title_list),ignore_index=True)

# print(title_list)

for link in Mb_URL_dic.values():
    driver.get(link)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    soup_title_list = soup.find_all('span', class_='spec-item')
    soup_data_list = soup.find_all('div', class_='spec-data')
    MB_name = soup.find('h1', class_='page-title').text

    title_list =['Board name']
    data_list = [MB_name]

    for title in soup_title_list:
        title_list.append(title.text)
    for data in soup_data_list:
        data_list.append(data.text)


    if soup_title_list :
        soup_data_list = soup.find_all('div', class_='spec-data')
        MB_name = soup.find('h1', class_='page-title').text
        data_list= [MB_name]
        for data in soup_data_list:
            data_list.append(data.text)
        print(data_list)
        try :
            df = df.append(pd.Series(data_list,index=title_list),ignore_index=True)
        except ValueError:
            print("Error MB page : " + MB_name)
    else :
        continue
df

['H110S1/CSM', ' Intel® for 6th Generation Core™ i7/Core™ i5/Core™ i3/Pentium®/Celeron® Processors Supports Intel® 14 nm CPUSupports Intel® Turbo Boost Technology 2.0 * The Intel® Turbo Boost Technology 2.0 support depends on the CPU types.* Refer to www.asus.com for CPU support list', ' Intel® H110', '2 x SO-DIMM, Max. 32GB, DDR4 2133 MHz Non-ECC, Un-buffered Memory Dual Channel Memory Architecture Supports Intel® Extreme Memory Profile (XMP)** Refer to www.asus.com for the Memory QVL (Qualified Vendors Lists).', 'Integrated Graphics Processor Multi-VGA output support : Dual HDMI/LVDS/DisplayPort   ports  - Supports HDMI with max. resolution 4096  x 2160  @ 24 Hz / 2560  x 1600  @ 60 Hz- Supports LVDS with max. resolution 1920  x 1200  @ 60  Hz - Supports DisplayPort with max. resolution 4906  x 2304  @ 60  Hz Maximum shared memory of 1024  MBSupports up to 2  displays simultaneously', 'Intel® H110 chipset : 1 x M.2 Socket 3, , with M key, type 2280 storage devices support (both SATA 

['H110T/CSM', ' Intel® for 6th Generation Core™ i7/Core™ i5/Core™ i3/Pentium®/Celeron® Processors Supports Intel® 14 nm CPUSupports Intel® Turbo Boost Technology 2.0 * The Intel® Turbo Boost Technology 2.0 support depends on the CPU types.* Refer to www.asus.com for CPU support list', ' Intel® H110', '2 x SO-DIMM, Max. 32GB, DDR4 2133 MHz Non-ECC, Un-buffered Memory Supports Intel® Extreme Memory Profile (XMP)** Refer to www.asus.com for the Memory QVL (Qualified Vendors Lists).', 'Integrated Graphics Processor- Intel® HD Graphics support Multi-VGA output support : HDMI/LVDS/DisplayPort   ports  - Supports HDMI with max. resolution 4096  x 2160  @ 24 Hz / 2560  x 1600  @ 60 Hz- Supports LVDS with max. resolution 1920  x 1200  @ 60  Hz - Supports DisplayPort with max. resolution 4096  x 2304  @ 60  Hz Maximum shared memory of 1024  MBSupports up to 2  displays simultaneously', 'Intel® H110 chipset : 1 x M.2 Socket 3, with M key, type 2242/2260 storage devices support (both SATA & PCIE m

['H110M-C/CSM', 'Build in Intel® for 6th Generation Core™ i7/Core™ i5/Core™ i3/Pentium®/Celeron® Processors Supports Intel® 14 nm CPUSupports Intel® Turbo Boost Technology 2.0 * The Intel® Turbo Boost Technology 2.0 support depends on the CPU types.* Refer to www.asus.com for CPU support list', 'Build in Intel® H110', '2 x DIMM, Max. 32GB, DDR4 2133 MHz Non-ECC, Un-buffered Memory Supports Intel® Extreme Memory Profile (XMP)** Refer to www.asus.com for the Memory QVL (Qualified Vendors Lists).', 'Integrated Graphics Processor- Intel® HD Graphics support Multi-VGA output support : DVI-D/RGB ports  - Supports DVI-D with max. resolution 1920  x 1200  @ 60 Hz - Supports RGB with max. resolution 1920  x 1200  @ 60  HzMaximum shared memory of 1024  MBSupports Intel® Quick Sync Video, Clear Video HD Technology', '1 x PCIe 3.0/2.0 x16  1 x PCIe 2.0 x1  1 x PCI  ', 'Intel® H110 chipset : 4 x SATA 6Gb/s port(s), gray, ', 'Realtek® RTL8111H, 1 x Gigabit LAN Controller(s)', 'Realtek® ALC887 8-Chan

['PRIME J3355I-C', 'Build in Intel®  Celeron® Dual-Core J3355 SoC onboard Processors ', '2 x U-DIMM, Max. 16GB, DDR3 1866*/1600/1333 MHz Non-ECC, Un-buffered Memory Dual Channel Memory Architecture ** Refer to www.asus.com for the Memory QVL (Qualified Vendors Lists).* The default memory operation frequency is dependent on its Serial Presence Detect (SPD), which is the standard way of accessing information from a memory module. Under the default state, some memory modules for overclocking may operate at a lower frequency than the vendor-marked value.* Due to the chipset limitation, we recommend that you use single-sided 4GB or 8GB, or double-sided 8GB DDR3 per DIMM slot. The maximum memory for the two DIMM slots is 16GB.', 'Integrated Graphics Processor Multi-VGA output support : HDMI/RGB/LVDS ports  - Supports HDMI with max. resolution 3840  x 2160  @ 30 Hz- Supports LVDS with max. resolution 1920  x 1200  @ 60  Hz - Supports RGB with max. resolution 1920  x 1200  @ 60  HzMaximum shar

['PRIME Z370-P', ' Intel® Socket 1151 for 8th Generation Core™  Processors Supports Intel® 14 nm CPUSupports Intel® Turbo Boost Technology 2.0 * Refer to www.asus.com for CPU support list', ' Intel® Z370', '4 x DIMM, Max. 64GB, DDR4 4000(O.C.)/3866(O.C.)/3733(O.C.)/3600(O.C.)/3466(O.C.)/3400(O.C.)/3333(O.C.)/3300(O.C.)/3200(O.C.)/3000(O.C.)/2800(O.C.)/2666/2400/2133 MHz Non-ECC, Un-buffered Memory *Dual Channel Memory Architecture Supports Intel® Extreme Memory Profile (XMP)** Refer to www.asus.com for the Memory QVL (Qualified Vendors Lists).* The maximum memory frequency supported varies by processor.', 'Integrated Graphics Processor- Intel® HD Graphics support Multi-VGA output support : HDMI/DVI-D ports - Supports HDMI 1.4b with max. resolution 4096  x 2160  @ 24  Hz / 2560  x 1600  @ 60  Hz - Supports DVI-D with max. resolution 1920  x 1200  @ 60 HzMaximum shared memory of 1024  MB', 'Supports AMD CrossFireX™ Technology ', '1 x PCIe 3.0/2.0 x16 (x16, x8/x4+x4) *1 x PCIe 3.0/2.0 x16

...

['Z10PE-D16 WS', 'Build in Intel® Socket 2011-3: Square type (80x80mm)  Processors Intel® Socket 2011-3 for Intel® Xeon® processor E5-2600 v3 product familyIntel® Socket 2011-3 for Intel® Xeon® processor E5-2600 v4 product family* Refer to www.asus.com for CPU support list', ' Intel® C612 PCH', '16 x DIMM, Max. 1024GB, DDR4 1866*/2400/2133/1600 MHz RDIMM, LR-DIMM Memory *1Quad Channel Memory Architecture ', 'ASpeed AST2400 32MB VGA port*2 - Supports VGA with max. resolution 1920  x 1200  @ 60  Hz', 'Supports NVIDIA® 3-Way SLI™ Technology Supports AMD Quad-GPU CrossFireX™ Technology ', '4 x PCIe 3.0/2.0 x16 (x16 mode) *32 x PCIe 3.0/2.0 x16 (x8 mode) *3', 'Intel® C612 chipset : 10 x SATA 6Gb/s port(s), gray, *4, 1 x M.2 Socket 3, black, *5, Support Raid 0, 1, 5, 10', 'Intel® I210-AT, 2 x Gigabit LAN Controller(s)1 x Management LAN', 'Realtek® ALC1150 8-Channel High Definition Audio CODEC - Supports : Jack-detection, Multi-streaming, Front Panel Jack-retasking- High quality 112 dB SNR st

['P10S-M WS', 'Build in Intel® Socket 1151 Core™ i7/Core™ i5/Core™ i3/for Xeon®  E3-1200 v5 and 6th genereation CoreTM, Pentium® and Celeron® Processors Supports Intel® 14 nm CPUSupports Intel® Turbo Boost Technology  * Refer to www.asus.com for CPU support list', 'Build in Intel® C236', '4 x DIMM, Max. 64GB, DDR4 2133 MHz ECC, Non-ECC, Un-buffered Memory Dual Channel Memory Architecture ** Refer to www.asus.com for the Memory QVL (Qualified Vendors Lists).', 'Integrated Graphics Processor Multi-VGA output support : HDMI/DVI-D/DisplayPort   ports  - Supports HDMI with max. resolution 4096  x 2160  @ 60 Hz / 4096  x 2160  @ 24 Hz - Supports DVI-D with max. resolution 1920  x 1200  @ 60 Hz - Supports DisplayPort with max. resolution 4096  x 2304  @ 60  Hz Maximum shared memory of 512  MBSupports Intel® HD Graphics, InTru™ 3D, Quick Sync Video, Clear Video HD Technology, Insider™', '1 x PCIe 3.0/2.0 x1 (x1 mode) *11 x PCIe 3.0/2.0 x16 (x16 mode) 1 x PCIe 3.0 x4 (x8 mode) *1', 'Intel® C236

['X99-WS/IPMI', 'Build in Intel® Socket 2011-v3 Core™ i7 Processors Supports Intel® 22 nm CPUSupports Intel® Turbo Boost Technology 2.0 * The Intel® Turbo Boost Technology 2.0 support depends on the CPU types.Intel® Socket 2011-v3 for Intel® Xeon® processor E5-1600 v3 product familyIntel® Socket 2011-v3 for Intel® Xeon® processor E5-2600 v3 product family', ' Intel® X99', '8 x DIMM, Max. 128GB, DDR4 3200(O.C.)/3000(O.C.)/2800(O.C.)/2666(O.C.)/2400(O.C.)/2133 MHz Non-ECC, Un-buffered Memory *18 x DIMM, Max. 128GB, DDR4 2133/1866* MHz ECC, Un-buffered Memory *28 x DIMM, Max. 128GB, DDR4 2133/1866* MHz ECC, Register Memory *2Quad Channel Memory Architecture Supports Intel® Extreme Memory Profile (XMP)* Hyper DIMM support is subject to the physical characteristics of individual CPUs.\xa0** Refer to www.asus.com for the Memory QVL (Qualified Vendors Lists).', 'Supports NVIDIA® 3-Way SLI™ Technology *3Supports AMD Quad-GPU CrossFireX™ Technology ', '40-Lane CPU-5 x PCIe 3.0/2.0 x16 (single a

['B250 MINING EXPERT', ' Intel® Socket 1151 for 7th/6th Generation Core™ i7/Core™ i5/Core™ i3/Pentium®/Celeron® Processors Supports Intel® 14 nm CPUSupports Intel® Turbo Boost Technology 2.0 * The Intel® Turbo Boost Technology 2.0 support depends on the CPU types.* Refer to www.asus.com for CPU support list', ' Intel® B250', '2 x DIMM, Max. 32GB, DDR4 2400/2133 MHz Non-ECC, Un-buffered Memory Supports Intel® Extreme Memory Profile (XMP)* Refer to www.asus.com for the Memory QVL (Qualified Vendors Lists).* Due to Intel® chipset limitation, DDR4 2400MHz memory frequency is only supported by 7th Generation Intel® processors. Higher memory modules will run at the maximum transfer rate of DDR4 2400MHz.** Due to Intel® chipset limitation, DDR4 2133MHz and higher memory modules on 6th Generation Intel® processors will run at the maximum transfer rate of DDR4 2133MHz.', 'Integrated Graphics Processor- Intel® HD Graphics support VGA output support : HDMI port - Supports HDMI with max. resolutio

,Accessories,Audio,BIOS,Back I/O Ports,Board name,CPU,Chipset,Form Factor,Graphic,Internal I/O Ports,...,OC Panel,System Bus,ThunderBolt LAN/Audio Combo Card,ThunderFX,Thunderbolt,Workstation Feature,Environment,OC Panel II,Weight,Warranty
0,User's manualI/O Shield1 x SATA power cable2 x...,Realtek® ALC3236 4-Channel High Definition Au...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, DMI 3.0,...",1 x DisplayPort1 x HDMI1 x LAN (RJ45) port(s)1...,H110S1/CSM,Intel® for 6th Generation Core™ i7/Core™ i5/C...,Intel® H110,Mini STX Form Factor 5.8 inch x 5.5 inch ( ...,Integrated Graphics Processor Multi-VGA output...,"1 x M.2 Socket 1 with E key, type 2230 Wi-Fi d...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,User's manualI/O ShieldThin Mini ITX I/O Shiel...,Realtek® ALC887-VD 8-Channel High Definition A...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, DMI 3.0,...",1 x DisplayPort1 x HDMI2 x LAN (RJ45) port(s)4...,Q170T/CSM,Build in Intel® for 6th Generation Core™ i7/Co...,Intel® Q170,Thin Mini ITX Form Factor 6.7 inch x 6.7 in...,Integrated Graphics Processor- Intel® HD Graph...,2 x USB 3.1 Gen 1 connector(s) support(s) addi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,User's manualI/O Shield2 x SATA 6Gb/s cable(s)...,Realtek® ALC887 7.1-Channel High Definition Au...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, DMI 3.0,...",1 x PS/2 keyboard/mouse combo port(s)1 x DVI-D...,H110I-PLUS/CSM,Intel® for 6th Generation Core™ i7/Core™ i5/C...,Intel® H110,Mini ITX Form Factor 6.7 inch x 6.7 inch ( ...,Integrated Graphics Processor- Intel® HD Graph...,1 x USB 3.1 Gen 1 connector(s) support(s) addi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,User's manualI/O ShieldThin Mini ITX I/O Shiel...,Realtek® ALC887 7.1-Channel High Definition Au...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, DMI 3.0,...",1 x DisplayPort1 x HDMI2 x LAN (RJ45) port(s)4...,H110T/CSM,Intel® for 6th Generation Core™ i7/Core™ i5/C...,Intel® H110,Thin Mini ITX Form Factor 6.7 inch x 6.7 in...,Integrated Graphics Processor- Intel® HD Graph...,3 x USB 2.0 connector(s) support(s) additional...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,User's manualI/O Shield2 x SATA 6Gb/s cable(s)...,Realtek® ALC887 8-Channel High Definition Audi...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, DMI3.0, ...",1 x PS/2 keyboard/mouse combo port(s)1 x DVI-D...,B150M-PLUS D3,Intel® Socket 1151 for 7th/6th Generation Cor...,Intel® B150,Micro ATX Form Factor 9.6 inch x 8.7 inch (...,Integrated Graphics Processor- Intel® HD Graph...,1 x USB 3.1 Gen 1 connector(s) support(s) addi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,User's manualI/O Shield2 x SATA 6Gb/s cable(s)...,Realtek® ALC887 8-Channel High Definition Audi...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, DMI2.0, ...",1 x PS/2 keyboard (purple)1 x PS/2 mouse (gree...,H110M-C/CSM,Build in Intel® for 6th Generation Core™ i7/Co...,Build in Intel® H110,9.6 inch x 7.2 inch ( 24.4 cm x 18.2 cm ),Integrated Graphics Processor- Intel® HD Graph...,1 x USB 3.1 Gen 1 connector(s) support(s) addi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,User's manualI/O Shield2 x SATA 6Gb/s cable(s)...,Realtek® ALC887 7.1-Channel High Definition Au...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, DMI3.0, ...",1 x PS/2 keyboard (purple)1 x PS/2 mouse (gree...,Q170M-C/CSM,Intel® for 6th Generation Core™ i7/Core™ i5/C...,Intel® Q170,Micro ATX Form Factor 9.6 inch x 8.9 inch (...,Integrated Graphics Processor- Intel® HD Graph...,1 x MONO out header 2 x USB 3.1 Gen 1 connecto...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,User's manualI/O Shield2 x SATA 6Gb/s cable(s)...,Realtek® ALC887 8-Channel High Definition Audi...,"128 Mb Flash ROM, UEFI AMI BIOS, PnP, DMI3.0, ...",1 x PS/2 keyboard (purple)1 x PS/2 mouse (gree...,PRIME Q270M-C/CSM,Intel® for 7th/6th Generation Core™ i7/Core™ ...,Intel® Q270,Micro ATX Form Factor,Integrated Graphics Processor Multi-VGA output...,1 x MONO out header *32 x USB 3.1 Gen 1 connec...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,User's manualI/O Shield2 x

In [27]:
##Dataframe to csv file

df.to_csv ("mb_spec.csv" , encoding = "utf-8")